# MSN news data set
----

1. The MIND dataset will be available at https://msnews.github.io.

2. The implementation of several existing news recommendation methods and general recommendation methods can be found at Microsoft Recommenders https://github.com/microsoft/recommenders

3. step-by-step tutorial 
https://towardsdatascience.com/a-step-by-step-guide-to-the-microsoft-news-recommendation-competition-700ab00831a


# Data Set
two folders for the same data
- MIND-small
    :: by randomly sampling 50,000 users and their behavior logs from the MIND dataset
    - training
    - validation 
- MIND-large
    - training
    - validation 
    - testing set

# Assignment Requirement
The assignment is to build a Recommender Syste that incorporates pluriformity and diversity to the algorithms and the metrics for an interactive news platform , using this data.  

Also in this final assignment we are extra attentive to the question of whether (how) the values, algorithmic affordances, and metrics align (check the elephant in the room slide of last lecture here) and to your empirical, technical and conceptual investigations for the concept.

There is also a folder for suggested readings on the challenges of news recommender system and diversity , that I would add some useful short readings to it. 


In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
# from torch.utils.data import *

import time
import random
import re

import matplotlib.pyplot as plt 
%matplotlib inline

import pdb
from tqdm.auto import tqdm

In [15]:
news = pd.read_csv("data/Small_data_set/MINDsmall_train/news.tsv", sep="\t", header=None)
behaviors = pd.read_csv("data/Small_data_set/MINDsmall_train/behaviors.tsv", sep="\t", header=None)

In [14]:
news.head(3)

,0,1,2,3,4,5,6,7
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."


In [16]:
behaviors.head(3)

,0,1,2,3,4
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
